In [12]:
import pandas as pd
import numpy as np
from collections import Counter 
#from Bio.Seq import Seq 
import itertools as itr 
import distance 
import datetime

Leemos el archivo:

In [2]:
%time st=pd.read_csv('/home/mb45296/bioinformatica/S04/Data/data_set.txt')

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 173 ms


In [3]:
st

,CACAGTAGGCGCCGGCACACACAGCCCCGGGCCCCGGGCCGCCCCGGGCCGGCGGCCGCCGGCGCCGGCACACCGGCACAGCCGTACCGGCACAGTAGTACCGGCCGGCCGGCACACCGGCACACCGGGTACACACCGGGGCGCACACACAGGCGGGCGCCGGGCCCCGGGCCGTACCGGGCCGCCGGCGGCCCACAGGCGCCGGCACAGTACCGGCACACACAGTAGCCCACACACAGGCGGGCGGTAGCCGGCGCACACACACACAGTAGGCGCACAGCCGCCCACACACACCGGCCGGCCGGCACAGGCGGGCGGGCGCACACACACCGGCACAGTAGTAGGCGGCCGGCGCACAGCC


In [2]:
dataset='CACAGTAGGCGCCGGCACACACAGCCCCGGGCCCCGGGCCGCCCCGGGCCGGCGGCCGCCGGCGCCGGCACACCGGCACAGCCGTACCGGCACAGTAGTACCGGCCGGCCGGCACACCGGCACACCGGGTACACACCGGGGCGCACACACAGGCGGGCGCCGGGCCCCGGGCCGTACCGGGCCGCCGGCGGCCCACAGGCGCCGGCACAGTACCGGCACACACAGTAGCCCACACACAGGCGGGCGGTAGCCGGCGCACACACACACAGTAGGCGCACAGCCGCCCACACACACCGGCCGGCCGGCACAGGCGGGCGGGCGCACACACACCGGCACAGTAGTAGGCGGCCGGCGCACAGCC'


In [5]:
len(dataset)

361

Ejercicio:

In [3]:
%time a=[dataset[i:i+10] for i in range(0, len(dataset), 1) if len(dataset[i:i+10]) ==10]
#Counter(dataset[i:i+10] for i in range(len(dataset)-9)) #se generan las secuencias de 10 posiciones dentro de 
#la cadena

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 963 µs


In [4]:
a

['CACAGTAGGC',
 'ACAGTAGGCG',
 'CAGTAGGCGC',
 'AGTAGGCGCC',
 'GTAGGCGCCG',
 'TAGGCGCCGG',
 'AGGCGCCGGC',
 'GGCGCCGGCA',
 'GCGCCGGCAC',
 'CGCCGGCACA',
 'GCCGGCACAC',
 'CCGGCACACA',
 'CGGCACACAC',
 'GGCACACACA',
 'GCACACACAG',
 'CACACACAGC',
 'ACACACAGCC',
 'CACACAGCCC',
 'ACACAGCCCC',
 'CACAGCCCCG',
 'ACAGCCCCGG',
 'CAGCCCCGGG',
 'AGCCCCGGGC',
 'GCCCCGGGCC',
 'CCCCGGGCCC',
 'CCCGGGCCCC',
 'CCGGGCCCCG',
 'CGGGCCCCGG',
 'GGGCCCCGGG',
 'GGCCCCGGGC',
 'GCCCCGGGCC',
 'CCCCGGGCCG',
 'CCCGGGCCGC',
 'CCGGGCCGCC',
 'CGGGCCGCCC',
 'GGGCCGCCCC',
 'GGCCGCCCCG',
 'GCCGCCCCGG',
 'CCGCCCCGGG',
 'CGCCCCGGGC',
 'GCCCCGGGCC',
 'CCCCGGGCCG',
 'CCCGGGCCGG',
 'CCGGGCCGGC',
 'CGGGCCGGCG',
 'GGGCCGGCGG',
 'GGCCGGCGGC',
 'GCCGGCGGCC',
 'CCGGCGGCCG',
 'CGGCGGCCGC',
 'GGCGGCCGCC',
 'GCGGCCGCCG',
 'CGGCCGCCGG',
 'GGCCGCCGGC',
 'GCCGCCGGCG',
 'CCGCCGGCGC',
 'CGCCGGCGCC',
 'GCCGGCGCCG',
 'CCGGCGCCGG',
 'CGGCGCCGGC',
 'GGCGCCGGCA',
 'GCGCCGGCAC',
 'CGCCGGCACA',
 'GCCGGCACAC',
 'CCGGCACACC',
 'CGGCACACCG',
 'GGCACACC

In [5]:
a=list(a)
a[0:5]

['CACAGTAGGC', 'ACAGTAGGCG', 'CAGTAGGCGC', 'AGTAGGCGCC', 'GTAGGCGCCG']

Existen 254 secuencias de 10 posiciones dentro de la cadena dataset

In [15]:
len(a)

352

debemos buscar aquella cadena k=10 posiciones que más se repite dentro de dataset con d=2 mismatches:
Paso1. generar todas las permutaciones de 10 posiciones del conjunto {A,C,G,T}

In [16]:
%time perms=list(itr.product(['A','C','G','T'],repeat=10))

CPU times: user 240 ms, sys: 8 ms, total: 248 ms
Wall time: 248 ms


In [17]:
for i in range(len(perms)):
    perms[i]="".join(perms[i])
perms[0:10]

['AAAAAAAAAA',
 'AAAAAAAAAC',
 'AAAAAAAAAG',
 'AAAAAAAAAT',
 'AAAAAAAACA',
 'AAAAAAAACC',
 'AAAAAAAACG',
 'AAAAAAAACT',
 'AAAAAAAAGA',
 'AAAAAAAAGC']

In [18]:
len(perms)

1048576

haciendo todas las permutaciones posibles tenemos $$ 4^5=1,048,576 $$ combinaciones, entonces para buscar la cadena de tamaño 10 que más aparece con a lo más d=2 mismatches, hacemos una matriz distancias de tamaño $$mxn$$ donde m es el número de permutaciones posibles y n es el número de secuencias en la cadena dataset:

Utilizando distancias de Hamming: $$d(AB,CB)=1$$ ya que si comparamos cada posición de las cadenas de texto difieren en la primera posición por tanto su disimiliridad o mismatch es 1, entonces para la matriz de distancias la entrada $$i,j$$ es la distancia de la permutación i vs. la secuencia j:   

In [19]:
import datetime

In [20]:
#Creamos la función que genra la matriz de distancias: 
def distx(L,W):
    k=np.zeros((len(L),len(W))) 
    for i in range(len(L)): 
        for j in range(len(W)):
            k[i,j]=distance.hamming(L[i],W[j])
    return(k) 

In [21]:
#Matriz de distancias
%time matdist=distx(perms,a)

CPU times: user 17min 28s, sys: 232 ms, total: 17min 28s
Wall time: 17min 28s


Tenemos entonces una matriz de tamaño 1,048,576 x 352, donde cada entrada representa la distancia de permutación vs. secuencia

In [22]:
np.shape(matdist)

(1048576, 352)

In [23]:
matdist

array([[  7.,   7.,   8., ...,   8.,   8.,   8.],
       [  6.,   7.,   7., ...,   8.,   7.,   7.],
       [  7.,   6.,   8., ...,   7.,   8.,   8.],
       ..., 
       [  8.,   9.,   8., ...,  10.,   9.,   9.],
       [  9.,   8.,   9., ...,   9.,  10.,  10.],
       [  9.,   9.,   9., ...,  10.,  10.,  10.]])

Dado que el renglón $$i$$ representa la $$i-ésima$$ permutación, entonces para saber cuantas veces se repite dentro de la secuencia con a lo más 2 mismatches  podemos sumar por renglón cuantas distancias son menores o iguales a 2 
$$
\sum_{j=1}^{352}x_{i,j}
$$
con $$ \left\{x_{i,j}=1 \ si \ d(i,j)<=2\right\} \ ó \ x_{i,j}=0 \ e.o.c. $$ 
entonces hacemos la función de sumas:

In [24]:
now=datetime.datetime.now()
##################################
freq=np.zeros(len(perms)) 
for k in range(len(perms)):
    freq[k]=sum(matdist[k,:]<=2)
####################################    
fin=datetime.datetime.now()
print(fin-now)

0:10:43.832382


Se genera el arreglo freq de tamaño 1,048,576  que contiene la frecuencia con que aparece la permutación:

In [25]:
freq

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [26]:
len(freq)

1048576

In [27]:
max(freq)

20.0

Se revisa cual es la cadena que más veces aparece y se obtiene la posición de la cadena que aparece más veces con el mismatch de 2

In [28]:
np.where(freq==max(freq))

(array([594209, 626961]),)

La cadena más frecuente aparece 13 veces, pero en este caso existen 2  cadenas:

In [29]:
print(perms[594209])
print(perms[626961])
#16064 CGGGCGGCGG 13
#46749 CGGCCGCCGG 13
#5803  GCGCACACAC 20
#      GCGCACACAC
#      GCGCACACAC
#52647 GCACACAGAC 20
#      GCACACAGAC

GCACACAGAC
GCGCACACAC


Para saber si la cadena esta o no dentro de la secuencia "dataset" podemos revisar la distancia mínima que existe entre la i-ésima permutación y las 254 secuencias de tal modo que si la distancia mínima es 0 entonces sabremos que la permutación está en la lista de secuencias de lo contrario no lo está:

In [30]:
min(matdist[594209,:]) #la cadena GCACACAGAC no está en la lista de secuencias

1.0

In [31]:
min(matdist[626961,:])#la cadena GCGCACACAC  está en la secuencia

0.0

In [32]:
distance.hamming(('C', 'G', 'G', 'C', 'C', 'G', 'C', 'C', 'G', 'G'),'CGGCCGCCGG')

0

In [1]:
#Creamos la función que genra la matriz de distancias: 
def distx(L,W):
    k=np.zeros((len(L),len(W))) 
    for i in range(len(L)): 
        for j in range(len(W)):
            k[i,j]=distance.hamming(L[i],W[j])
    return(k) 


def kmers(cadena,k,d):
    #generamos todas los strings de tamaño k dentro de la cadena
    a=list([cadena[i:i+k] for i in range(0, len(cadena), 1) if len(cadena[i:i+k]) ==k]) 
    #se generan todas las permutaciones de tamalo k que se pueden hacer con el conjunto {A,C,G,T} 
    perms=list(itr.product(['A','C','G','T'],repeat=k))
    #perms es de la forma ['A','C','G','T'] y lo transformamos a la forma 'ACGT'
    for w in range(len(perms)):
        perms[w]="".join(perms[w])
    #se crea la matriz de distancias de permutaciones y secuencias de tamaño k de la cadena
    matdist=distx(perms,a)
    #contamos los renglones(permutaciones) que tienen distancia<=2 y generamos el vector freq con los conteos
    freq=np.zeros(len(perms)) 
    for j in range(len(perms)):
        freq[j]=sum(matdist[j,:]<=d)
    #Se revisa cual es la permutación que más veces aparece con mismatch d y 
    #se obtiene la posición donde se encuentra
    v=np.where(freq==max(freq))[0]
    for n in range(len(v)):
        print(perms[v[n]]," ",freq[v[n]])
            

In [14]:
now=datetime.datetime.now()
dataset=dataset[0:20]
kmers(dataset,10,2) 
        
fin=datetime.datetime.now()
print("tiempo total: ",fin-now)

GCACACAGAC   20.0
GCGCACACAC   20.0
tiempo total:  0:27:48.222125


In [67]:
v=np.where(freq==max(freq))[0]

In [68]:
len(v)

2

In [69]:
print("tiempo total: ",fin-now)

tiempo total:  0:11:44.119673


In [9]:
perms[0]

'AAAAAAAAAA'